#  Geospatial Analytics - Lesson 10: Routing on road networks and SUMO tools

Author: Giuliano Cornacchia
</br>Geospatial Analytics, Master degree in Data Science and Business Informatics, University of Pisa

___
In this lesson, we will learn how to handle and use SUMO tools in Python and from command line.

1. [sumolib](#sumolib)
2. [duarouter](#duarouter)
3. [TraCi](#traci)
4. [Routing](#routing)

<a id='sumolib'></a>
## sumolib

sumolib is a python library for working with SUMO networks.

ref: https://sumo.dlr.de/docs/Tools/Sumolib.html

In [1]:
# import the sumolib library
import sumolib

# read the road network
net = sumolib.net.readNet('road_network_pisa.net.xml')

In [2]:
# get the list of edges
edge_list = net.getEdges()

# get the list of nodes
node_list = net.getNodes()

print(f"The road network has {len(edge_list)} edges and {len(node_list)} nodes.")

The road network has 5436 edges and 2695 nodes.


In [3]:
# get the coordinate of an edge (we consider the from node)

e = edge_list[42]

x, y = e.getFromNode().getCoord()
lon, lat = net.convertXY2LonLat(x, y)

print(f"SUMO reference coordinates (x,y): {x,y}")
print(f"GPS coordinates (lon,lat): {lon, lat}")

SUMO reference coordinates (x,y): (6401.3, 8556.03)
GPS coordinates (lon,lat): (10.430917392312276, 43.721093738985964)


#### Retrieve the closest edge to a GPS point

In [4]:
lon_polo_fibonacci, lat_polo_fibonacci = 10.407580310750713, 43.72082373893976

# convert into SUMO coordinates (x, y)
x_polo_fibonacci, y_polo_fibonacci = net.convertLonLat2XY(lon_polo_fibonacci, lat_polo_fibonacci)
print(f"SUMO reference coordinates (x,y): {x_polo_fibonacci,y_polo_fibonacci}")

SUMO reference coordinates (x,y): (4522.0103448751615, 8493.85369954072)


In [5]:
# retrieve the set of Neighboring edges
candiates_edges = net.getNeighboringEdges(x_polo_fibonacci, y_polo_fibonacci, r=25)

# IMPORTANT! the result of net.getNeighboringEdges is not sorted by distance
edges_and_dist = sorted(candiates_edges, key = lambda x: x[1])
edges_and_dist

[(<edge id="313284660#0" from="303595336" to="246382226"/>, 17.52380688345678),
 (<edge id="-313284660#0" from="246382226" to="303595336"/>,
  20.72474590039968)]

In [6]:
closest_edge_fibonacci = edges_and_dist[0][0]

print(f"Name: {closest_edge_fibonacci.getName()}")
print(f"Edge ID: {closest_edge_fibonacci.getID()}")

Name: Via Filippo Buonarroti
Edge ID: 313284660#0


#### Compute the shortest and the fastest path between two edges

In [7]:
# get the shortest path (minimizes length) and fastest path (minimzes travel time)

e_origin, e_destination = e, closest_edge_fibonacci

In [8]:
# shortest path

shortest_path = net.getOptimalPath(e_origin, e_destination, fastest=False)

print(f"Number of edges in the path: {len(shortest_path[0])}")
print(f"Cost [m]: {shortest_path[1]}")

Number of edges in the path: 48
Cost [m]: 2182.6500000000005


In [9]:
# fastest path

fastest_path = net.getOptimalPath(e_origin, e_destination, fastest=True)

print(f"Number of edges in the path: {len(fastest_path[0])}")
print(f"Cost [s]: {fastest_path[1]}")

Number of edges in the path: 48
Cost [s]: 163.008742192388


<a id='duarouter'></a>
## duarouter

Duarouter is a python tool that computes vehicle routes that may be used by SUMO.

In other words, duarouter converts **Incomplete Routes** into **Routes** according to a specified routing algorithm.

ref: https://sumo.dlr.de/docs/duarouter.html

In [10]:
import subprocess

# prepare the command string for duarouter

command_str = "duarouter --route-files traffic_demand_pisa.rou.xml "+\
        " --net-file road_network_pisa.net.xml"+\
        " --output-file traffic_demand_pisa_duarouter.rou.xml"

Let's call the duarouter python application

In [11]:
p = subprocess.Popen(command_str, shell=True, stdout=subprocess.PIPE, 
                                     stderr=subprocess.STDOUT)
retval = p.wait()

Look in the current folder, you will find the outputs of duarouter.
- `traffic_demand_pisa_duarouter.rou.xml`: the generated traffic demand file with vehicles and routes;`
- `traffic_demand_pisa_duarouter.rou.alt.xml`: additionally, a `.rou.alt.xml` file will be generated; it contains route ditribution.



Duarouter supports different routing algorithms (e.g., dijkstra and A*).

You can specify which algorithm to use using the option `--routing-algorithm <ALG>`

see https://sumo.dlr.de/docs/Simulation/Routing.html#routing_algorithms

In [12]:
command_str_astar = "duarouter --route-files traffic_demand_pisa.rou.xml "+\
        " --net-file road_network_pisa.net.xml"+\
        " --output-file traffic_demand_pisa_duarouter_astar.rou.xml --routing-algorithm astar"

p = subprocess.Popen(command_str_astar, shell=True, stdout=subprocess.PIPE, 
                                     stderr=subprocess.STDOUT)
retval = p.wait()

#### random perturbation

Duarouter allows perturbing the fastest path using a randomisation parameter
𝑤 ∈ \[1, $+\infty$\), where 𝑤 = 1 means no randomisation (i.e., the fastest path), and the higher 𝑤, the more randomly perturbed the fastest
path is.

You can specify the degree of perturbation ($w$) with the option `--weights.random-factor <WEIGHT>`

<img src="https://i.ibb.co/9bc7yb5/Screenshot-2022-11-16-at-14-24-58-How-Routing-Strategies-Impact-Urban-Emissions-How-Routing-Strategi.png" width="300px" border="0">

In [13]:
command_str_w = "duarouter --route-files traffic_demand_pisa.rou.xml "+\
        " --net-file road_network_pisa.net.xml"+\
        " --output-file traffic_demand_pisa_duarouter_w.rou.xml --weights.random-factor 300"

p = subprocess.Popen(command_str_w, shell=True, stdout=subprocess.PIPE, 
                                     stderr=subprocess.STDOUT)
retval = p.wait()

<a id="traci"></a>
## TraCi

TraCi is a Python controller that allows to control and manipulate at runtime any aspect of the simulation, e.g., the internal data structures of SUMO; the cost to pay is an overhead.
With TraCi is possible to retrieve several simulations variables (e.g., vehicles' GPS positions, vehicles' emissions and so on). <br>
TraCi allows us to make the vehicles communicate or coordinate with themselves and many more. <br><br>
See https://sumo.dlr.de/docs/TraCI.html#value_retrieval for a list of variable you can extract with TraCi.

ref: https://sumo.dlr.de/docs/TraCI.html

In [14]:
# import the traci library
import traci
from utils import init_traci

In [15]:
# initialize TraCi by specifing the configuration file

init_traci("./sumo_config_pisa.sumocfg")

n_steps = 10

# simulate each step
for step in range(n_steps):
    
    # perform a simulation step
    traci.simulationStep()
    
    # get the list of active vehicles (vehicles inserted in the simulation)
    vehicle_list = traci.vehicle.getIDList()
    
    # value retrieval
    for v_id in vehicle_list:
        
        # some examples of value you can retrieve
        
        # Speed [m/s]
        speed_ms = traci.vehicle.getSpeed(v_id)
        
        # CO2 emissions [mg/s] 
        co2_emissions = traci.vehicle.getCO2Emission(v_id)
        
        # Position
        x, y = traci.vehicle.getPosition(v_id)
        lon, lat = traci.simulation.convertGeo(x, y)
        
        print(f"Vehicle ID: {v_id}")
        print(f"position [lon, lat]: {lon, lat}")
        print(f"speed [m/s]: {speed_ms}")
        print(f"CO2 emissions [mg/s]: {co2_emissions}\n")
        
        
# close TraCi when the total number of steps to simulate is reached
traci.close()

Vehicle ID: flow_0.0
position [lon, lat]: (10.430849628417304, 43.721135474381505)
speed [m/s]: 1.8798571447376162
CO2 emissions [mg/s]: 3329.7118399280153

Vehicle ID: flow_osm.0
position [lon, lat]: (10.429803034985646, 43.721654024739436)
speed [m/s]: 1.4338497789110989
CO2 emissions [mg/s]: 2990.7303919361098

Vehicle ID: flow_0.0
position [lon, lat]: (10.430807539029978, 43.721151336329065)
speed [m/s]: 3.8211460547521714
CO2 emissions [mg/s]: 4177.159503894869

Vehicle ID: flow_osm.0
position [lon, lat]: (10.429829708429862, 43.72168475206212)
speed [m/s]: 4.033343723951839
CO2 emissions [mg/s]: 4962.098029013955

Vehicle ID: flow_0.0
position [lon, lat]: (10.430738713378283, 43.72117727414366)
speed [m/s]: 6.248432177538052
CO2 emissions [mg/s]: 6073.936609609667

Vehicle ID: flow_osm.0
position [lon, lat]: (10.429870752710245, 43.72173203425156)
speed [m/s]: 6.206377868866548
CO2 emissions [mg/s]: 5637.345415400625

Vehicle ID: flow_0.0
position [lon, lat]: (10.430650199876556,

<a id="routing"></a>
## Routing

So far we have seen three ways to define routes (routing strategies) between an origin and destination edges:<br>
1. Shortest Path;<br>
2. Fastest Path;<br>
3. Random perturbation of 1 or 2.<br><br>

However, GPS navigation apps such as TomTom, Google Maps,
and Waze, which use routing algorithms, heuristics and AI to suggest the best path to reach a user’s desired destination should be consider to generate realist traffic scenarios.<br>
In this section of the notebook, we will see how to query OpenStreetMap (OSM) and how to make a SUMO vehicle follow the path suggested by OSM.
___

### OpenStreetMap routing API

For obtaining a **key** you need to register at https://openrouteservice.org/dev/#/signup.

In the Free plan, you have **2000 free requests daily**. Once registered, on the web profile there is a counter of the daily requests made (the "free non-tile requests" in the "Activity" section).

See https://openrouteservice.org/dev/#/api-docs/directions for the routing parameters that affect the routing algorithm.

The **most important parameter** is `preference`, which specifies the type of optimization used when calculating routes (can be `fastest`, `shortest`, `recommended`). Default is `recommended`.
- `fastest`: Route calculation is optimized by travel time, while keeping the routes sensible. For example, the calculation may avoid shortcuts along inconvenient side roads or long detours that only save very little time.
- `shortest`: Route calculation is optimized by travel distance, while keeping the routes sensible. For example, straight routes are preferred over those incurring turns.
- `recommended`: Route calculation is optimized susing an internal OpenStreetMap algorithm that takes into account both distance and travel time.

In our analysis we use `recommended`.

Other parameters are, for example: 
- `profile` (`driving-car`, `cycling-regular`, `foot-walking`, ...). Default is `driving-car`.
- `geometry_simplify`

In [16]:
# coordinates of origin and destination (using sumolib)

# origin
x_origin, y_origin = e_origin.getFromNode().getCoord()
lon_origin, lat_origin = net.convertXY2LonLat(x_origin, y_origin)

# destination
x_dest, y_dest = e_destination.getFromNode().getCoord()
lon_dest, lat_dest = net.convertXY2LonLat(x_dest, y_dest)

coordinates = [[lon_origin, lat_origin], [lon_dest, lat_dest]]

In [17]:
# import OSM
import openrouteservice as ors


#init the connection
client = ors.Client(key="<YOUR-KEY>")

# make the query to OSM
route_osm = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        preference="recommended",
        format='geojson',
        geometry_simplify=False,
        validate=True)

# extract the GPS points (lon, lat) of the suggested route
gps_points_osm = [coord for coord in route_osm['features'][0]['geometry']['coordinates']]

In [18]:
# visualize the suggestion on a folium map

import folium

m = folium.Map(location=[lat_origin, lon_origin], tiles='cartodbpositron', zoom_start=13)

# reverse the GPS points into (lat, lon) for folium
folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in gps_points_osm]).add_to(m)
m

### Map-Matching (in a very naive way)

To make a vehicle follow the sequence of GPS points suggested from OSM we need to "translate" the sequence of GPS point into a sequence of SUMO edges.

A very naive way is to assign to each GPS point the closest SUMO edge.

In [21]:
import itertools

edges_osm = []

for p in gps_points_osm:
    
    # we use sumolib to get the closest edge
    # project p (lon, lat) into SUMO coordinates (x, y)
    x, y = net.convertLonLat2XY(p[0], p[1])
    
    # retrieve the set of Neighboring edges
    candiates_edges = net.getNeighboringEdges(x, y, r=25)

    # IMPORTANT! the result of net.getNeighboringEdges is not sorted by distance
    edges_and_dist = sorted(candiates_edges, key = lambda x: x[1])
    
    # take the closest edge ID
    matched_edge = edges_and_dist[0][0].getID()
    
    # append the edge to the list
    edges_osm.append(matched_edge)

In [22]:
# Hint: to be more robust we can consider only "pivot" edges, i.e, edges that are matched at least three consecutive times

pivot_edges = [(k, sum(1 for _ in vs)) for k, vs in itertools.groupby(edges_osm)]
pivot_edges = [elem[0] for elem in pivot_edges if elem[1]>2]

We can copy (by hand for now) the list of edges into the traffic demand file.
See the flow with ID "flow_osm" in the traffic demand file "traffic_demand_pisa.rou.xml" in the folder of this notebook